# Shows how to Use the Data Alan Created and Use Utility Functions

In [65]:
import pickle
from collections import namedtuple
import pandas as pd
import numpy as np
from importlib import reload
import sys

# bench_util module is in the parent directory, because it will be part of
# the production code.  Add the parent directory to the Path so Python can
# import from there
sys.path.insert(0, '../')   
import bench_util

## Unpickle the Dataframe for Preprocessed Utility Data & Make Utility Function Object

In [66]:
# Unpickle the pre-processed DataFrame
df = pickle.load(open('dfu3.pkl', 'rb'))

# Unpickle the raw utility bill DataFrame, which is needed below to make
# the utility function object.
df_raw = pickle.load(open('df_raw.pkl', 'rb'))

df.head()  # the processed data

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,03,Electricity,2005,12,Electricity charge,kWh,1904.657880,49.5,14790.748577,2006,6,50.466034
1,03,Electricity,2006,1,Electricity charge,kWh,5430.493797,93.0,42665.790911,2006,7,145.575679
2,03,Electricity,2006,2,Electricity charge,kWh,5764.406730,84.0,45010.439348,2006,8,153.575619
3,03,Electricity,2006,3,Electricity charge,kWh,6349.255299,93.0,46311.547557,2006,9,158.015000
4,03,Electricity,2006,4,Electricity charge,kWh,5529.385224,90.0,40392.812893,2006,10,137.820278


In [94]:
# this is only needed to update any code changes I may have made
# since last importing the module above.
reload(bench_util)     

# Make an object that has the various utility functions.
# The object needs access to the raw utility bill DataFrame and the spreadsheet
# containing other application data.
ut = bench_util.Util(df_raw, '../data/Other_Building_Data.xlsx')

In [92]:
# Check Totals
print(df_raw.Cost.sum(), df.cost.sum())
print(df_raw.Usage.sum(), df.usage.sum())

109128189.09000163 109128189.08999836
958777495.4999995 958777495.5000011


## DataFrame for "Utility Cost Overview" Report Page

In [68]:
site = 'ANSBG1'
# From the main DataFrame, get only the rows for this site, and only get
# the needed columns for this analysis
df1 = df.query('site_id == @site')[['service_type', 'fiscal_year', 'fiscal_mo', 'cost']]
df1.head()

,service_type,fiscal_year,fiscal_mo,cost
30353,Electricity,2006,12,739.280
30354,Electricity,2006,12,0.000
30355,Electricity,2006,12,183.065
30356,Electricity,2007,1,1414.275
30357,Electricity,2007,1,0.000


In [69]:
df2 = pd.pivot_table(
    df1, 
    values='cost', 
    index=['fiscal_year'], 
    columns=['service_type'],
    aggfunc=np.sum
)
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water
fiscal_year,,,,,
2006,922.345000,NaN,NaN,NaN,NaN
2007,24319.055469,NaN,NaN,3886.897857,3224.555714
2008,33274.530938,NaN,NaN,5401.766996,5085.043697
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167


In [70]:
# Add in columns for the missing services
missing_services = bench_util.missing_services(df2.columns)
bench_util.add_columns(df2, missing_services)
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam
fiscal_year,,,,,,,
2006,922.345000,NaN,NaN,NaN,NaN,0.0,0.0
2007,24319.055469,NaN,NaN,3886.897857,3224.555714,0.0,0.0
2008,33274.530938,NaN,NaN,5401.766996,5085.043697,0.0,0.0
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588,0.0,0.0
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0


In [71]:
# Add a Total column that sums the other columns
df2['Total'] = df2.sum(axis=1)
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam,Total
fiscal_year,,,,,,,,
2006,922.345000,NaN,NaN,NaN,NaN,0.0,0.0,922.345000
2007,24319.055469,NaN,NaN,3886.897857,3224.555714,0.0,0.0,31430.509040
2008,33274.530938,NaN,NaN,5401.766996,5085.043697,0.0,0.0,43761.341631
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588,0.0,0.0,146092.058918
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0,142213.157351
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0,162227.108988
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0,147059.389193
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0,126884.959788
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0,94026.849736


In [72]:
# Add a percent change column
df2['pct_change'] = df2.Total.pct_change()
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam,Total,pct_change
fiscal_year,,,,,,,,,
2006,922.345000,NaN,NaN,NaN,NaN,0.0,0.0,922.345000,NaN
2007,24319.055469,NaN,NaN,3886.897857,3224.555714,0.0,0.0,31430.509040,33.076738
2008,33274.530938,NaN,NaN,5401.766996,5085.043697,0.0,0.0,43761.341631,0.392320
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588,0.0,0.0,146092.058918,2.338382
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0,142213.157351,-0.026551
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0,162227.108988,0.140732
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0,147059.389193,-0.093497
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0,126884.959788,-0.137186
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0,94026.849736,-0.258960


In [73]:
# Add in degree days
months_present = bench_util.months_present(df1)
deg_days = ut.degree_days_yearly(months_present, site)
df2['hdd'] = deg_days
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam,Total,pct_change,hdd
fiscal_year,,,,,,,,,,
2006,922.345000,NaN,NaN,NaN,NaN,0.0,0.0,922.345000,NaN,NaN
2007,24319.055469,NaN,NaN,3886.897857,3224.555714,0.0,0.0,31430.509040,33.076738,NaN
2008,33274.530938,NaN,NaN,5401.766996,5085.043697,0.0,0.0,43761.341631,0.392320,13802.0
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588,0.0,0.0,146092.058918,2.338382,12393.0
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0,142213.157351,-0.026551,12171.0
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0,162227.108988,0.140732,13802.0
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0,147059.389193,-0.093497,12393.0
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0,126884.959788,-0.137186,12171.0
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0,94026.849736,-0.258960,13802.0


In [74]:
# Add in a column to show the numbers of months present for each year
# This will help to identify partial years.
mo_count = bench_util.month_count(months_present)
df2['month_count'] = mo_count
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2006,922.345000,NaN,NaN,NaN,NaN,0.0,0.0,922.345000,NaN,NaN,1
2007,24319.055469,NaN,NaN,3886.897857,3224.555714,0.0,0.0,31430.509040,33.076738,NaN,12
2008,33274.530938,NaN,NaN,5401.766996,5085.043697,0.0,0.0,43761.341631,0.392320,13802.0,12
2009,30307.350563,100896.902333,3592.6,5823.032433,5472.173588,0.0,0.0,146092.058918,2.338382,12393.0,12
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0,142213.157351,-0.026551,12171.0,12
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0,162227.108988,0.140732,13802.0,12
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0,147059.389193,-0.093497,12393.0,12
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0,126884.959788,-0.137186,12171.0,12
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0,94026.849736,-0.258960,13802.0,12


In [75]:
# Reverse the DataFrame
df2.sort_index(ascending=False, inplace=True)
df2

service_type,Electricity,Natural Gas,Oil #1,Sewer,Water,Refuse,Steam,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2018,7256.446970,3779.601452,NaN,1236.853750,1046.692321,0.0,0.0,13319.594493,-0.873651,NaN,3
2017,54161.057879,41955.989017,NaN,4978.546250,4323.157679,0.0,0.0,105418.750824,0.181032,13802.0,12
2016,32391.098485,45647.201628,NaN,6108.227500,5113.362500,0.0,0.0,89259.890113,-0.049376,12171.0,12
2015,38017.419167,47128.677258,NaN,4593.016167,4156.942333,0.0,0.0,93896.054925,-0.001391,12393.0,12
2014,35843.822155,48967.142581,NaN,4840.116833,4375.768167,0.0,0.0,94026.849736,-0.258960,13802.0,12
2013,36961.451595,79970.329194,NaN,5169.324500,4783.854500,0.0,0.0,126884.959788,-0.137186,12171.0,12
2012,41994.697841,94490.242371,NaN,5484.028148,5090.420833,0.0,0.0,147059.389193,-0.093497,12393.0,12
2011,35138.080909,117477.010810,NaN,4971.323638,4640.693631,0.0,0.0,162227.108988,0.140732,13802.0,12
2010,32688.293030,99742.203356,NaN,5074.845929,4707.815036,0.0,0.0,142213.157351,-0.026551,12171.0,12


In [76]:
# Export the rows of the DataFrame into a list of dictionaries for use
# in the template.  Rename columns to match template variables.
row_dicts = bench_util.df_to_dictionaries(df2, change_names={'Natural Gas': 'natural_gas', 'Oil #1': 'fuel_oil'})
row_dicts

[{'Electricity': 7256.4469696969691,
  'Refuse': 0.0,
  'Sewer': 1236.85375,
  'Steam': 0.0,
  'Total': 13319.594492738443,
  'Water': 1046.6923214285714,
  'fiscal_year': 2018,
  'fuel_oil': nan,
  'hdd': nan,
  'month_count': 3.0,
  'natural_gas': 3779.601451612903,
  'pct_change': -0.87365061349557049},
 {'Electricity': 54161.057878787855,
  'Refuse': 0.0,
  'Sewer': 4978.5462500000003,
  'Steam': 0.0,
  'Total': 105418.75082449638,
  'Water': 4323.1576785714287,
  'fiscal_year': 2017,
  'fuel_oil': nan,
  'hdd': 13802.0,
  'month_count': 12.0,
  'natural_gas': 41955.989017137101,
  'pct_change': 0.18103159987302431},
 {'Electricity': 32391.098484848484,
  'Refuse': 0.0,
  'Sewer': 6108.2274999999991,
  'Steam': 0.0,
  'Total': 89259.890112872687,
  'Water': 5113.3625000000011,
  'fiscal_year': 2016,
  'fuel_oil': nan,
  'hdd': 12171.0,
  'month_count': 12.0,
  'natural_gas': 45647.201628024202,
  'pct_change': -0.049375501617985096},
 {'Electricity': 38017.419166666674,
  'Refuse':

In [77]:
# Put results into the final report dictionary that will be passed to the Template.
# This assumes you have created a couple graphs with the file names shown below.
pg3 = dict(rows=row_dicts, 
           g1='output/images/{}_pg2_g1.png'.format(site), 
           g2='output/images/{}_pg2_g2.png'.format(site)
      )
pg3

{'g1': 'output/images/ANSBG1_pg2_g1.png',
 'g2': 'output/images/ANSBG1_pg2_g2.png',
 'rows': [{'Electricity': 7256.4469696969691,
   'Refuse': 0.0,
   'Sewer': 1236.85375,
   'Steam': 0.0,
   'Total': 13319.594492738443,
   'Water': 1046.6923214285714,
   'fiscal_year': 2018,
   'fuel_oil': nan,
   'hdd': nan,
   'month_count': 3.0,
   'natural_gas': 3779.601451612903,
   'pct_change': -0.87365061349557049},
  {'Electricity': 54161.057878787855,
   'Refuse': 0.0,
   'Sewer': 4978.5462500000003,
   'Steam': 0.0,
   'Total': 105418.75082449638,
   'Water': 4323.1576785714287,
   'fiscal_year': 2017,
   'fuel_oil': nan,
   'hdd': 13802.0,
   'month_count': 12.0,
   'natural_gas': 41955.989017137101,
   'pct_change': 0.18103159987302431},
  {'Electricity': 32391.098484848484,
   'Refuse': 0.0,
   'Sewer': 6108.2274999999991,
   'Steam': 0.0,
   'Total': 89259.890112872687,
   'Water': 5113.3625000000011,
   'fiscal_year': 2016,
   'fuel_oil': nan,
   'hdd': 12171.0,
   'month_count': 12.0,

## Fiscal Month Abbreviations

In [78]:
bench_util.mo_list

['Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec',
 'Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun']

In [79]:
bench_util.mo_dict

{1: 'Jul',
 2: 'Aug',
 3: 'Sep',
 4: 'Oct',
 5: 'Nov',
 6: 'Dec',
 7: 'Jan',
 8: 'Feb',
 9: 'Mar',
 10: 'Apr',
 11: 'May',
 12: 'Jun'}

## Get Additional Information on a Building

In [80]:
# Request the building using the Site ID
bldg = ut.building_info('ANSBG1')
bldg

BldgInfo(site_name='FNSB-Animal Control', site_category='Other', address='2408 Davis Road', city='Fairbanks', primary_func='Animal Shelter', year_built=nan, sq_ft=6718.0, onsite_gen=nan, dd_site='PAFA', source_elec='Golden Valley Electric', source_oil='Sourdough Fuel  (Petro Star)', source_nat_gas='Fairbanks Natural Gas', source_steam='', source_water='Golden Heart Utilities', source_sewer='Golden Heart Utilities', source_refuse='', acct_elec='31850', acct_oil='25465', acct_nat_gas='10282  (2408 DAVIS)', acct_steam='', acct_water='1311001100', acct_sewer='1311001100', acct_refuse='')

In [81]:
# Shows some fields of info about the building
bldg.site_name, bldg.sq_ft, bldg.dd_site

('FNSB-Animal Control', 6718.0, 'PAFA')

In [82]:
bldg._asdict()

OrderedDict([('site_name', 'FNSB-Animal Control'),
             ('site_category', 'Other'),
             ('address', '2408 Davis Road'),
             ('city', 'Fairbanks'),
             ('primary_func', 'Animal Shelter'),
             ('year_built', nan),
             ('sq_ft', 6718.0),
             ('onsite_gen', nan),
             ('dd_site', 'PAFA'),
             ('source_elec', 'Golden Valley Electric'),
             ('source_oil', 'Sourdough Fuel  (Petro Star)'),
             ('source_nat_gas', 'Fairbanks Natural Gas'),
             ('source_steam', ''),
             ('source_water', 'Golden Heart Utilities'),
             ('source_sewer', 'Golden Heart Utilities'),
             ('source_refuse', ''),
             ('acct_elec', '31850'),
             ('acct_oil', '25465'),
             ('acct_nat_gas', '10282  (2408 DAVIS)'),
             ('acct_steam', ''),
             ('acct_water', '1311001100'),
             ('acct_sewer', '1311001100'),
             ('acct_refuse', '')])

In [83]:
# Instead of building info for one building, get a DataFrame with
# all the building information
ut.building_info_df().head()

,acct_elec,acct_nat_gas,acct_oil,acct_refuse,acct_sewer,acct_steam,acct_water,address,city,dd_site,...,site_name,source_elec,source_nat_gas,source_oil,source_refuse,source_sewer,source_steam,source_water,sq_ft,year_built
site_id,,,,,,,,,,,,,,,,,,,,,
03,172230-5,10283 - DENALI,55010001,AW23-DNL03,1998005610,,"1998005610, 1998073600",NaN,NaN,PAFA,...,SD-Denali Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,49210.0,NaN
04,172995-3,10283 - NORDALE,55017001,AW16-NDL04,2440271010,,2440271010,NaN,NaN,PAFA,...,SD-Nordale Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,49210.0,NaN
05,"104270-4, 104271-2, 321383, 357683, 577004",,55016001,AW17-LTH05,1918027500,"08-112-0, 08-113-0",1918027500,NaN,NaN,PAFA,...,SD-Lathrop High,Golden Valley Electric,,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,Aurora Energy,Golden Heart Utilities,234412.0,NaN
06,112356-1,10283 - HUNTER,55012001,AW21-HNT06,1995007000,,1995007000,NaN,NaN,PAFA,...,SD-Hunter Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,57047.0,NaN
07,76471-2,10283-U-PARK,55028001,AW04-UPK07,4012700710,,4012700710,NaN,NaN,PAFA,...,SD-University Park Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,College Utilities,,College Utilities,64699.0,NaN


In [84]:
# Get a list of all Site IDs, sorted alphabetically
ut.all_sites()

['03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '104',
 '11',
 '12',
 '13',
 '14',
 '15',
 '15A',
 '15B',
 '16',
 '20',
 '22',
 '23',
 '27',
 '28',
 '29',
 '34',
 '36',
 '37',
 '38',
 '39',
 '40',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '49',
 '76',
 'ADLER',
 'ANSBG1',
 'ASLC18',
 'ASLC21',
 'ASLC47',
 'ASLC52',
 'ASLCHU',
 'ASLCV1',
 'ASLELC1',
 'ASLELC2',
 'ASLGDM',
 'ASLGP2',
 'ASLHIS',
 'ASLPIH',
 'ASLPL1',
 'ASLSEA',
 'ASLSQD',
 'ASLTVR',
 'BALHHW',
 'BAOBG1',
 'BAP001',
 'BENBG1',
 'BHPBHG',
 'BHPBHL',
 'BHPBHW',
 'BHPCCS',
 'BHPSKI1',
 'BHPSKI2',
 'BHPSKI3',
 'BHPSKI4',
 'CACBG1',
 'CBS001',
 'CEC',
 'CLX001',
 'CLX002',
 'CLX003',
 'CLX004',
 'CLXES1',
 'CLXGP2',
 'CLXSM1',
 'CLXSO1',
 'CRB001',
 'CSP001',
 'DIPMP1',
 'DOGPRK',
 'Emer S T',
 'GF001',
 'GFP001',
 'GRP001',
 'GRPLFT',
 'GSWNP',
 'HEMBG1',
 'HEZ001',
 'HSPSWP',
 'KEP001',
 'KIP001',
 'KWP001',
 'LEABG1',
 'LF001',
 'MF001',
 'MNPPRK',
 'MSLL001',
 'MSP001',
 'MSRSWP',
 'MSWBG1',
 'MSWBG2',
 'MSWWAR',


In [95]:
ut.site_categories_and_buildings()

[{'name': 'Other',
  'sites': [{'id': 'ADLER', 'name': 'FNSB-Adler School building'},
   {'id': 'ANSBG1', 'name': 'FNSB-Animal Control'},
   {'id': 'BAP001', 'name': 'FNSB-Bernice Aldridge Park'},
   {'id': 'DIPMP1', 'name': 'FNSB-Big Dipper'},
   {'id': 'BHPBHG', 'name': 'FNSB-Birch Hill Garage'},
   {'id': 'BHPCCS', 'name': 'FNSB-Birch Hill Ski Building'},
   {'id': 'BHPSKI1', 'name': 'FNSB-Birch Hill Ski Trail #1'},
   {'id': 'BHPSKI2', 'name': 'FNSB-Birch Hill Ski Trail #2'},
   {'id': 'BHPSKI3', 'name': 'FNSB-Birch Hill Ski Trail #3'},
   {'id': 'BHPSKI4', 'name': 'FNSB-Birch Hill Ski Trail #4'},
   {'id': 'BHPBHL', 'name': 'FNSB-Birch Hill Timing Building'},
   {'id': 'BHPBHW', 'name': 'FNSB-Birch Hill Warm-up Building'},
   {'id': 'CLX001', 'name': 'FNSB-CLRA Chena Lakes Rec Area'},
   {'id': 'CLX002', 'name': 'FNSB-CLRA Chena Lakes Rec Park'},
   {'id': 'CLX004', 'name': 'FNSB-CLRA Chena River Lakes Proj A'},
   {'id': 'CLX003', 'name': 'FNSB-CLRA Chena River Lakes Rec Proj'},


In [86]:
# Convert the notebook to a script.
# !jupyter nbconvert --to script use_alan_data.ipynb

## Deal with Divide by Zero

In [87]:
df_test = df.head().copy()
df_test['div0'] = df_test.mmbtu / 0.0
df_test

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu,div0
0,03,Electricity,2005,12,Electricity charge,kWh,1904.657880,49.5,14790.748577,2006,6,50.466034,inf
1,03,Electricity,2006,1,Electricity charge,kWh,5430.493797,93.0,42665.790911,2006,7,145.575679,inf
2,03,Electricity,2006,2,Electricity charge,kWh,5764.406730,84.0,45010.439348,2006,8,153.575619,inf
3,03,Electricity,2006,3,Electricity charge,kWh,6349.255299,93.0,46311.547557,2006,9,158.015000,inf
4,03,Electricity,2006,4,Electricity charge,kWh,5529.385224,90.0,40392.812893,2006,10,137.820278,inf


In [88]:
print(np.isfinite(np.nan))
print(np.isfinite(np.inf))
print(np.isfinite(-np.inf))
print(np.isfinite(2.34))

False
False
False
True


In [89]:
np.isfinite(None)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
def is_blank(val):
    return not np.isfinite(val)

In [ ]:
print(is_blank(-np.inf))
print(is_blank(np.inf))
print(is_blank(np.nan))
print(is_blank(2.3))